In [ ]:
import os
import tqdm
import imageio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt          

from numpy import linalg as LA
from scipy import optimize
from itertools import cycle
from sklearn import preprocessing,svm, metrics
from umap import UMAP
from skimage import io

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm
from sklearn.datasets import load_digits,load_iris
from mlxtend.plotting import plot_decision_regions
from sklearn.cluster import MiniBatchKMeans,MeanShift
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings; warnings.simplefilter('ignore')

### Punto 1 - Preprocesamiento de datos

In [ ]:
df = pd.read_csv("sign_mnist_train.csv")
df.head()
df.shape
letter2encode = {'A': 0,'B': 1,'C': 2,'D': 3,'E': 4,'F': 5,'G': 6,'H': 7,'I': 8,'K': 9,'L': 10,'M': 11,
                'N': 12,'O': 13,'P': 14,'Q': 15,'R': 16,'S': 17,'T': 18,'U': 19,'V': 20,'W': 21,'X': 22, 'Y': 23}

def fix_label_gap(l):
    if(l>=9):
        return (l-1)
    else:
        return l

def encode(character):
    return letter2encode[character]

df['label'] = df['label'].apply(fix_label_gap)
WORD = 'THANKS'
word = np.array(list(WORD))
embedded_word = list(map(encode, word))
reduced_df = df[df['label'].isin(embedded_word)]
reduced_df.shape
X = reduced_df.loc[:, reduced_df.columns != 'label'].values
y = reduced_df['label'].values
plt.imshow(X[12].reshape(28,28))
X_PCA = PCA(n_components=5).fit_transform(X)
X_LDA = LDA(n_components=5).fit_transform(X,y)
X_TSNE = TSNE().fit_transform(X)
X_UMAP = UMAP(n_neighbors=15,
                      min_dist=0.1,
                      metric='correlation').fit_transform(X)

fig = plt.figure(figsize=(50,40))
plt.subplot(2,2,1)
plt.scatter(X_PCA[:,0], X_PCA[:,1], c=y, cmap='Set1')
plt.title("Principal Component Analysis", fontsize=40)
plt.subplot(2,2,2)
plt.scatter(X_UMAP[:,0], X_UMAP[:,1], c=y, cmap='Set1')
plt.title("Uniform Manifold Approximation and Projections", fontsize=40)
plt.subplot(2,2,3)
plt.scatter(X_LDA[:,0], X_LDA[:,1], c=y, cmap='Set1')
plt.title("Linear Discriminant Analysis", fontsize=40)
plt.subplot(2,2,4)
plt.scatter(X_TSNE[:,0], X_TSNE[:,1], c=y, cmap='Set1')
plt.title("t-Distributed Stochastic Neighbor Embedding", fontsize=40)
plt.show()

#### ¿Cuántos componentes se deben usar para explicar la variabilidad del 70% y 80% de los datos en PCA?

In [ ]:

#Cargamos un dataset de 64 columnas
digits = load_digits()

#reducimos a 2 dimensiones el dataset
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)

#ploteamos el resultado
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
# Ahora observamos la varianza de los datos con la cantidad total de columnas del dataset (64 columnas)
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Numero de columnas')
plt.ylabel('Varianza acumulada');

Esta curva cuantifica qué parte de la varianza total de 64 dimensiones está contenida dentro de las primeras columnas. Por ejemplo, vemos que con el dataset original, las primeras 10 columnas contienen aproximadamente el 75% de la varianza, mientras que se necesitan conservar alrededor de 50 columnas para describir cerca del 100% de la varianza.

### Punto 2-Regresion

In [ ]:
dataset = pd.read_excel('Concrete_Data.xls')
df = dataset.drop(['Concrete compressive strength(MPa, megapascals) '],axis=1)

x = dataset.iloc[:, 0:7]
y = dataset.iloc[:, 7]
#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(x.values, y.values, test_size=0.2)
from sklearn.tree import DecisionTreeRegressor
adr = DecisionTreeRegressor(max_depth = 5)
adr.fit(X_train, y_train)

#Realizo una predicción
Y_pred = adr.predict(X_test)
from sklearn import tree
from subprocess import check_call
from IPython.display import Image as PImage

# exportar el modelo a archivo .dot
with open(r"tree1.dot", 'w') as f:
     f = tree.export_graphviz(adr,
                              out_file=f,
                              max_depth = 7,
                              impurity = True,
                              feature_names = list(df)[:7],
                              rounded = True,
                              filled= True )
        
# Convertir el archivo .dot a png para poder visualizarlo
check_call(['dot','-Tpng',r'tree1.dot','-o',r'tree1.png'])
PImage("tree1.png")


In [ ]:
print('Coefficient of determination: %.2f'% metrics.r2_score(y_test, Y_pred))
print('Mean squared error: %.2f' % metrics.mean_squared_error(y_test, Y_pred))

### Punto 3-Clasificacion

In [ ]:
iris = datasets.load_iris()
X = iris.data[:, 2:]  # we only take the last two features.
y = iris.target
C = 1.0  # SVM regularization parameter
h=.02
# SVC with linear kernel
svc = svm.SVC(kernel='linear', C=C).fit(X, y)
# LinearSVC (linear kernel)
lin_svc = svm.LinearSVC(C=C).fit(X, y)
# SVC with RBF kernel
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
# SVC with polynomial (degree 3) kernel
poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
np.arange(y_min, y_max, h))

# title for the plots
titles = ['SVC with linear kernel','LinearSVC (linear kernel)','SVC with RBF kernel','SVC with polynomial (degree 3) kernel']

for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.subplot(2, 2, i + 1)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
 
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
 
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
 
    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
    plt.xlabel('Petal length')
    plt.ylabel('Petal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(titles[i])
plt.show()

Es preferible optar por un kernel SVM lineal si se tiene una gran cantidad de atributos en los datos(> 1000) porque es más probable que los datos se puedan separar linealmente en un espacio de muchas dimensiones. tambien se puede usar RBF, pero siempre se debe realizar una validación cruzada de entre los parámetros  de configuracion para evitar un ajuste excesivo.

En el caso del grafico de RBF se ve una mejor separacion de los datos debido a que aumenta el area de los puntos rojos y se reducen las areas tanto azul como celeste indicando que el algoritmo puede identificar mucho mejor a los grupos y a que grupo pertenece cada punto del dataset.

#### Calidad Vino

In [ ]:
#UTILIZANDO UN SVC LINEAL
df_vinos = pd.read_csv('calidad_vino.csv', sep=';')
X = df_vinos.drop(['quality'], axis=1) #dejamos las features solamente
y = df_vinos['quality'] #dejamos el target
xs = X.values
ys = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y)
svclassifier_vinos = svm.SVC(kernel='linear', gamma='scale', probability=True)  # Kernel lineal
svclassifier_vinos.fit(X_train, y_train) 
y_pred = svclassifier_vinos.predict(X_test)
pred_proba = svclassifier_vinos.predict_proba(X_test)

#METRICAS
print ("f1 score macro", metrics.f1_score(y_test, y_pred, average='macro'))
print("log loss", metrics.log_loss(y_test, pred_proba))
print ("accuracy",metrics.accuracy_score(y_test, y_pred))
print ("precision score",metrics.precision_score(y_test, y_pred,average='macro')) 
#print("AUC",metrics.roc_auc_score(y_test, y_pred)) #TODO

In [ ]:
#UTILIZANDO UN SVC con nucleo RBF
df_vinos = pd.read_csv('calidad_vino.csv', sep=';')
X = df_vinos.drop(['quality'], axis=1) #dejamos las features solamente
y = df_vinos['quality'] #dejamos el target
xs = X.values
ys = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y)
svclassifier_vinos = svm.SVC(kernel='rbf', gamma='scale', probability=True)  # Kernel lineal
svclassifier_vinos.fit(X_train, y_train) 
y_pred = svclassifier_vinos.predict(X_test)
pred_proba = svclassifier_vinos.predict_proba(X_test)

#METRICAS
print ("f1 score macro", metrics.f1_score(y_test, y_pred, average='macro'))
print("log loss", metrics.log_loss(y_test, pred_proba))
print ("accuracy",metrics.accuracy_score(y_test, y_pred))
print ("precision score",metrics.precision_score(y_test, y_pred,average='macro')) 
#print("AUC",metrics.roc_auc_score(y_test, y_pred)) #TODO

### Analisis de resultados

In [ ]:
#todo

In [ ]:
# Valores promedio para todas las columnas cuando se busca la calidad 6 en los vinos
df_vinos[df_vinos.quality == 6].describe().iloc[[1]]

### Area de decision

In [ ]:
# Fit Support Vector Machine Classifier
df_vinos = pd.read_csv('calidad_vino.csv', sep=';')
#X = df_vinos.drop(['quality'], axis=1) #dejamos las features solamente
X = df_vinos.drop(df_vinos.columns[[2,3,4,5,6,7,8,9,10,11]], axis=1) #dejamos solo los dos atributos de acidez
y = df_vinos['quality'] #dejamos el target

clf = svm.SVC(kernel='linear',decision_function_shape='ovo')
clf.fit(X.values, y.values) 

# Plot Decision Region using mlxtend's awesome plotting function
plot_decision_regions(X=X.values, y=y.values, clf=clf, legend=2)

# Update plot object with X/Y axis labels and Figure Title
plt.xlabel(X.columns[0], size=14)
plt.ylabel(X.columns[1], size=14)
plt.title('SVM Decision Region Boundary', size=16)

### Punto 4 - Clustering

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    
    # choose a random subset
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20);

In [ ]:
foto = io.imread('colores.jpg') #TODO agregar un control para jupyter
data = foto / 255.0
data = data.reshape(foto.shape[0] * foto.shape[1], 3)

In [ ]:
kmeans = MiniBatchKMeans(80)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

In [ ]:
foto_recolored = new_colors.reshape(foto.shape)
fig, ax = plt.subplots(1, 2, figsize=(16, 6), subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(foto)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(foto_recolored)
ax[1].set_title('80-color Image', size=16);

Este algoritmo se puede utilizar para reducir el tamaño de imagenes y puede utilizarse para cargar imagenes cuando la conexion es de baja velocidad o para hacer gifs o para imagenes del tipo thumbnail.

In [ ]:
plot_pixels(data, title='Espacio de colores de la imagen original')

In [ ]:
plot_pixels(data, colors=new_colors,title="Reduced color space: 80 colors")

In [ ]:
dataset = load_iris()
ms = MeanShift()
ms.fit(dataset.data)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_clusters_ = labels.max()+1
plt.figure(1)
plt.clf()
colors = cycle('rcmy')

for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(dataset.data[my_members, 0], dataset.data[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1],
             'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Cant. de clusters : %d' % n_clusters_)
plt.show()

### Punto 5 - Métodos de Ensambles

In [ ]:
#USANDO RANDOM FOREST COMO METODO DE CLASIFICACION DE ENSAMBLE
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
# create the classifier
classifier = RandomForestClassifier(n_estimators=100)

# Train the model using the training sets
classifier.fit(X_train, y_train)
# predictin on the test set
y_pred = classifier.predict(X_test)
# Calculate Model Accuracy
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
classifier = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, predictions))
print(metrics.confusion_matrix(y_test, predictions))

A simple vista el metodo de ensamble que utiliza el algoritmo **RandomForestClassifier** tiene un mayor nivel de
confiabilidad con respecto al algoritmo **AdaBoostClassifier** siendo que ambos trabajan sobre el mismo dataset.
Ademas se observa que la suma de los elementos de la diagonal de la matriz de confusion del algoritmo **RandomForestClassifier** da un mayor valor que la del segundo algoritmo indicando que pudo identificar mejor a los elementos de las clases.